# SVM 모델 실습
## 핸드폰 가격 예측 모델 만들기
*데이터 : https://www.kaggle.com/iabhishekofficial/mobile-price-classification?select=train.csv

*코드 참고 : https://chancoding.tistory.com/67

### 1. 라이브러리

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

### 2. 데이터 불러오기 및 확인

In [ ]:
df = pd.read_csv(r"   ")  # 본인 파일 주소 넣으시면 됩니다.
test=pd.read_csv(r"   ")   # 본인 파일 주소 넣으시면 됩니다.

#데이터의 기본 구조 확인
df.head() 

In [ ]:
# 결측치 확인
#결측값(missing value)는 분석 오류를 발생시키거나 왜곡시킬 위험이 있습니다.
#따라서 분석할 DataFrame을 생성하고나면 결측값 여부를 확인합니다.

df.isnull().sum()  

In [ ]:
#데이터 프레임 기본 상태 확인
df.info()

In [ ]:
#가격을 나타내는 target value 확인 
#우리는 "핸드폰 가격"을 예측하려고 합니다. 
#해당 데이터는 0, 1, 2, 3 네 개의 class를 가진 카테고리형 데이터입니다.

df['price_range'].unique()

### 3. SVM 모델 생성   (ppt 20p)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# feature 와 target 분리

# target은 우리의 target 데이터인 "price_range"
yt=np.array(df['price_range']) 

# feature는 target 데이터를 뺀 나머지 데이터 전부
xt=df.drop(['price_range'], axis=1) 
xt=np.array(xt)

In [ ]:
# scaler : 각 Feature의 값을 일정한 범위 또는 규칙에 따르게 하기 위해서 스케일링을 사용
# MinMaxScaler : 최소 0 ~ 최대 1 사이로 feature의 값을 변환

scaler = MinMaxScaler()
xt=scaler.fit_transform(xt)

In [ ]:
# train test 데이터 split
# 훈련용 데이터인 'train.csv' 의 데이터 값들을 다시 훈련과 검증용 데이터로 split 해야함
# x_train :  data 값
# y_train : target 값 (price_range)
# x_valid : data 값
# y_valid : target 값 (price_range)
# 훈련용 : 검증용 데이터를 8 : 2 로 분리  --> test_size = 0.2

xtrain,xtest,ytrain,ytest=train_test_split(xt, yt, test_size=0.2, random_state=42)

### 3-1. linear kernel  SVC를 통해 적절한 C 찾기  (ppt 22p)

#### C
- C 는 SVM의 기본 파라미터
- 슬랙 변수의 가중치
- 분류를 하는데 오차를 허용하는 범위인 마진 값을 조정함
- C 값이 커질수록 오분류 허용 X
- C 값이 작아지면 margin 폭 좁아짐.
- C 값이 커지면 margin 폭 넓어짐.

In [ ]:
# Linear 커널 SVC

scores = []
for thisC in [*range(1,50)]:
    svc = SVC(kernel='linear', C = thisC)  #linear kernel SVC의 파라미터 : kernel , C
    model = svc.fit(xtrain, ytrain)
    scoreTrain = model.score(xtrain, ytrain)
    scoreTest = model.score(xtest,ytest)
    print("Linear SVM : C :{}, training score:{:2f}, test score:{:2f}".format(thisC, scoreTrain, scoreTest))
scores.append([scoreTrain, scoreTest])

### 3-2. rbf kernel 사용한 SVC를 통해 적절한 C 와 gamma 찾기  (ppt 23p)

#### C
- C 는 SVM의 기본 파라미터
- 슬랙 변수의 가중치
- 분류를 하는데 오차를 허용하는 범위인 마진 값을 조정함
- C 값이 커질수록 오분류 허용 X
- C 값이 작아지면 margin 폭 좁아짐.
- C 값이 커지면 margin 폭 넓어짐

#### gamma
- 가우시안 함수의 표준편차와 관련있는 매개변수
- gamma는 RBF 커널에서 하나의 데이터 샘플이 영향력을 행사하는 거리를 나타냄
- gamma가 크면 데이터 포인터들이 영향력을 행사하는 거리가 짧아짐
- gamma가 작으면 영향력을 행사하는 거리가 커짐
- gamma가 커질수록 곡률 높음

In [ ]:
# RBF 커널 SVM
for thisGamma in [.1, .25, .5, 1] :  # gamma 값을 [ 0.1 , 0.25 , 0.5 , 1 ]로 다양하게 지정
    for thisC in [ 1, 5, 10, 20, 40, 100] : # C 값을 [ 1, 5, 10, 20, 40, 100 ] 으로 다양하게 지정
        model2 = SVC(kernel="rbf", C=thisC, gamma = thisGamma).fit(xtrain, ytrain) #linear kernel SVC의 파라미터 : kernel , C , gamma
        m2train = model2.score(xtrain, ytrain)
        m2test = model2.score(xtest, ytest)
        print("RBF SVM : C:{}, gamma:{},training score:{:2f},test score:{:2f} \n".format(thisC, thisGamma, m2train, m2test))

##### 하지만 c 와 gamma를 어느 세월에 다 찾을 수 있을까?

### 3-3. GridSearchCV를 사용 (ppt 25p)

최적의 파라미터와 그 때의 score 값을 한 번에 찾아줌

In [ ]:
from sklearn.model_selection import GridSearchCV
param={'C':[1,5,10,20,40,100],
      'gamma':[.1, .25, .5, 1]}
GS=GridSearchCV(SVC(kernel='rbf'),param, cv=5)
GS.fit(xtrain, ytrain)
print(GS.best_params_)
print(GS.best_score_)

### 4. 예측

test에 적용 후 최종 예측 값 얻기

In [ ]:
test=test.drop(['id'],axis=1)
test.head()
testmat=np.array(test)
test=scaler.fit_transform(test)
#test(DF -> array)
model=SVC(kernel='rbf', C=5, gamma=.1).fit(xtrain, ytrain)

In [ ]:
prediction=model.predict(test)
pred=pd.DataFrame(prediction)
pred

*수고하셨습니다*